# Data sharing - iRODS permissions

In this tutorial we will show you how to share your data with other users in the iRODS system. To see the effects it is helpful to team up with a colleague!!!

## What are permissions

Permissions in iRODS are part of the data objects and collections. They give information and they allow other users of the same iRODS system to read and edit your data.

In iRODS there are four types of permissions:

| Access | Meaning|
|:---:|:---|
|read| The user or the group can **read and download** the data.|
|write| The user or group can **modify data objects and upload data to collections**, but can not delete the data or collection.|
|own| The user or group can **share and delete** the data.|

## Give access to data objects and collections

In [2]:
from ibridges.interactive import interactive_auth
session = interactive_auth()

Auth without password


Let us have a look at the permissions of our demo data:

In [8]:
from ibridges import get_collection, get_dataobject
from ibridges.path import IrodsPath

irods_coll_path = IrodsPath(session, '~').joinpath('demo')
coll = get_collection(session, irods_coll_path)
obj = get_dataobject(session, irods_coll_path.joinpath('demofile.txt'))

Like for the metadata we need to load the permissions.

In [10]:
from ibridges.permissions import Permissions
coll_perm = Permissions(session, coll)
print(coll_perm)

datamanager-its#nluu12p
	read object	rodsgroup
read-test-christine#nluu12p
	read object	rodsgroup
research-test-christine#nluu12p
	own	rodsgroup
u.aksu@uu.nl#nluu12p
	read object	rodsuser
inheritance True



In a native iRODS instance you will see your own username who is labeled with `own`. In Yoda you will see three groups: *research-\<group\>*, *read-\<group\>* and the *\<datamanager\>* group. Every user in these groups has the respective permissions on the data.

Now we iwll give your colleague explicit access. **Note that if your colleague is also in one of the respective Yoda groups, the highest permission (the most powerful) will apply.**

In [14]:
coll_perm.set('write', '<username>')
print(coll_perm)

datamanager-its#nluu12p
	read object	rodsgroup
read-test-christine#nluu12p
	read object	rodsgroup
research-test-christine#nluu12p
	own	rodsgroup
u.aksu@uu.nl#nluu12p
	modify object	rodsuser
inheritance True



You will see a new entry with your colleague's iRODS name and *modify object*. We can get the default permission names like that:

In [15]:
coll_perm.available_permissions

OrderedDict([('null', 1000),
             ('read object', 1050),
             ('modify object', 1120),
             ('own', 1200)])

Note that some permission-types have synonyms:

+ read object: 'read', 'read object', 'read_object'
+ modify object: 'write', 'modify object', 'modify_object'

## Remove access

To remove any permissions, we use the keyword *null*. Again, please note, if your colleague is still part of a group that has access to your data, this will have no effect.

In [16]:
coll_perm.set('null', '<username>')
print(coll_perm)

datamanager-its#nluu12p
	read object	rodsgroup
read-test-christine#nluu12p
	read object	rodsgroup
research-test-christine#nluu12p
	own	rodsgroup
inheritance True



iBridges also prohibits that you lock yourself out of your data:

In [18]:
coll_perm.set('write', session.username)

ValueError: Cannot set your own permissions, because you would lose access to the object/collection.

All the above functionality can also be used for objects

In [19]:
obj_perm = Permissions(session, obj)
print(obj_perm)

datamanager-its#nluu12p
	read object	rodsgroup
	read object	rodsgroup
read-test-christine#nluu12p
	read object	rodsgroup
	read object	rodsgroup
research-test-christine#nluu12p
	own	rodsgroup
	own	rodsgroup
rods#nluu12p
	own	rodsadmin
	own	rodsadmin
u.aksu@uu.nl#nluu12p
	read object	rodsuser
	read object	rodsuser



## Collections: Inheritance

You might have noticed that collections carry a special permissions keyword *inheritance* which can be `True` or `False`. If inheritance is switched on all new data objects and subcollections will receive the same permissions as the parent collection.

Below we create a new collection, ans give access to your colleague and switch te inheritance on:

In [25]:
new_coll_path = IrodsPath(session, '~').joinpath('demo1')
coll = IrodsPath.create_collection(session, new_coll_path)
coll_perm = Permissions(session, coll)
coll_perm.set('write', 'u.aksu@uu.nl')
coll_perm.set('inherit')
print(coll_perm)

datamanager-its#nluu12p
	read object	rodsgroup
read-test-christine#nluu12p
	read object	rodsgroup
research-test-christine#nluu12p
	own	rodsgroup
u.aksu@uu.nl#nluu12p
	modify object	rodsuser
inheritance True



Now we create a new subcollection:

In [26]:
subcoll = IrodsPath.create_collection(session, new_coll_path.joinpath('subcoll'))

And inspect the permissions. They should be the same as the ones above.

In [28]:
subcoll_perm = Permissions(session, subcoll)
print(subcoll_perm)

False
datamanager-its#nluu12p
	read object	rodsgroup
read-test-christine#nluu12p
	read object	rodsgroup
research-test-christine#nluu12p
	own	rodsgroup
u.aksu@uu.nl#nluu12p
	modify object	rodsuser
inheritance True



Now we will switch off the inheritance of the parent collection and create another subcollection:

In [29]:
coll_perm.set('noinherit')
print(coll_perm)

datamanager-its#nluu12p
	read object	rodsgroup
read-test-christine#nluu12p
	read object	rodsgroup
research-test-christine#nluu12p
	own	rodsgroup
u.aksu@uu.nl#nluu12p
	modify object	rodsuser
inheritance False



In [30]:
subcoll = IrodsPath.create_collection(session, new_coll_path.joinpath('subcoll1'))
subcoll_perm = Permissions(session, subcoll)
print(subcoll_perm)

c.staiger@uu.nl#nluu12p
	own	rodsuser
inheritance False



You will see now that you `own` *subcoll1*, that